■ 仕様
現在レート、4時間平均レート、24時間平均レート、過去25営業日の最高値と最安値、過去25営業日の最高値と最安値から何日経過したかで、現在の相場をパターン分けし過去10年の売買データから適切（適切な値は遺伝的アルゴリズムで計算して過去のデータから求める）な注文を行う。

■ 動作例
通貨:ドル円
現在時刻: 2015年10月1日 10:00:00
現在レート: 120.00円
4時間平均レート: 119.80円
24時間平均レート: 119.40円
過去25営業日の最高値: 121.75円
過去25営業日の最安値: 117.25円
過去25営業日の最高値からの経過日数: 2日
過去25営業日の最安値からの経過日数: 8日
注文内容: 120円で100単位BUY、利益確定は120.60円、損切りは119.75円


In [2]:
# -*- coding: utf-8 -*-
import random
import datetime
from enum import Enum

def order(ai_order):
    _base = "OrderDone:{}, Limit:{}, Stop-Limit:{}"
    print _base.format(ai_order.order_type,
                       ai_order.limit,
                       ai_order.stop_limit)


class OrderType(Enum):
    Buy = 1
    Wait = 0
    Sell = -1


class Rate(object):
    start_at = datetime.datetime(2015, 10, 1, 10, 0, 0)
    bid = float(120.00)  # 現在のレート
    h4 = float(119.80)  # 過去4時間のレート平均
    h24 = float(119.40)  # 過去24時間のレート平均
    low_25d = float(117.25)  # 過去25営業日の最安値
    high_25d = float(121.75)  # 過去25営業日の最高値
    low_25d_passed_day = 8  # 過去25営業日の最安値から何日経過したか
    high_25d_passed_day = 2  # 過去25営業日の最高値から何日経過したか
    base_tick = float(0.01)  # 1tickあたりの最小売買単位

    @classmethod
    def get(cls, currency=None, start_at=None):
        return Rate()

    def get_rate(self, tick, is_add):
        if is_add:
            return self.bid + float(tick * self.base_tick)
        else:
            return self.bid - float(tick * self.base_tick)


class AIOrder(object):
    def __init__(self, order_type, limit, stop_limit):
        self.order_type = order_type
        self.limit = limit
        self.stop_limit = stop_limit


class MarketKeyMixin(object):
    def get_key_h24(self):
        """
        現在レートと過去24時間平均の差分を表すkeyを返却
        例) H24:5
        :rtype: str
        """
        point = (self.rate.bid - self.rate.h24) / self.rate.base_tick
        return 'H24:{}'.format(int(point / self.standard_tick))

    def get_key_h4(self):
        """
        現在レートと過去4時間平均の差分を表すkeyを返却
        例) H4:0
        :rtype: str
        """
        point = (self.rate.bid - self.rate.h4) / self.rate.base_tick
        return 'H4:{}'.format(int(point / self.standard_tick))

    def get_key_low_25d(self):
        """
        現在レートと過去25営業日中の最低値との乖離を表すkeyを返却
        例) LOW:3
        :rtype: str
        """
        point = (self.rate.bid - self.rate.low_25d) / self.rate.base_tick
        return 'LOW:{}'.format(int(point / self.standard_tick))

    def get_key_high_25d(self):
        """
        現在レートと過去25営業日中の最高値との乖離を表すkeyを返却
        例) HIGH:2
        :rtype: str
        """
        point = (self.rate.bid - self.rate.high_25d) / self.rate.base_tick
        return 'HIGH:{}'.format(int(point / self.standard_tick))

    def get_key_low_passed_day(self):
        """
        過去25営業日中の最低値から何日経過したか
        例) LOW_PASSED_DAY:10
        :rtype: str
        """
        return 'LOW_PASSED_DAY:{}'.format(self.rate.low_25d_passed_day)

    def get_key_high_passed_day(self):
        """
        過去25営業日中の最高値から何日経過したか
        例) HIGH_PASSED_DAY:1
        :rtype: str
        """
        return 'HIGH_PASSED_DAY:{}'.format(self.rate.high_25d_passed_day)


class AIParamValue(object):
    """
    AIの動作データを定義するクラス
    """
    def __init__(self, value, _min, _max):
        self.value = value
        self._min = _min
        self._max = _max

    def __repr__(self):
        return "VALUE:{},MIN:{},MAX:{}".format(self.value,
                                               self._min,
                                               self._max)

    def mutation(self):
        """
        突然変異
        """
        self.value = random.randint(self._min, self._max)
        return self


class AIParamOrder(object):
    """
    AIの動作データのうち発注時の動作を定義するクラス
    """
    def __init__(self, order_type, limit, stop_limit, _min, _max):
        self.order_type = order_type
        self.limit = limit
        self.stop_limit = stop_limit
        self._min = _min
        self._max = _max

    def __repr__(self):
        _base = "ORDER-TYPE:{},LIMIT:{},STOP-LIMIT:{},MIN:{},MAX:{}"
        return _base.format(self.order_type,
                            self.limit,
                            self.stop_limit,
                            self._min,
                            self._max)

    def mutation(self):
        """
        突然変異
        """
        self.order_type = random.choice(list(OrderType))
        self.limit = random.randint(self._min, self._max)
        self.stop_limit = random.randint(self._min, self._max)
        return self


class AI(MarketKeyMixin):
    DEFAULT_STANDARD_TICK = 30
    DEFAULT_MIN = 15
    DEFAULT_MAX = 120
    # AIの動作データを保存
    param = {
        'DEFAULT_STANDARD_TICK': AIParamValue(DEFAULT_STANDARD_TICK, DEFAULT_MIN, DEFAULT_MAX)
    }

    def __init__(self, rate):
        self.rate = rate

    def order(self):
        """
        発注する。
        """
        order(self._get_order())

    def get_key(self):
        """
        Rateを解析・分類し、適切なKEYを生成する

        # sample
        'H24:5,H4:0,LOW:4,LOW_PASSED_DAY:3,HIGH:3,HIGH_PASSED_DAY:6'
        """
        _base = [self.get_key_h24(),
                 self.get_key_h4(),
                 self.get_key_low_25d(),
                 self.get_key_high_25d(),
                 self.get_key_low_passed_day(),
                 self.get_key_high_passed_day()]
        return ",".join(_base)

    def _get_order(self):
        """
        発注クラスを返却
        rtype: AIOrder
        """
        # Rateを解析する
        key = self.get_key()
        _o = self.param.get(key, None)
        if _o is None:
            # AIの注文データが存在しないときはランダム生成して保存
            _o = self._get_random_order()
            self.param[key] = _o

        # 発注データの生成
        limit_rate = self.rate.get_rate(_o.limit, True)
        stop_limit_rate = self.rate.get_rate(_o.stop_limit, False)
        return AIOrder(_o.order_type, limit_rate, stop_limit_rate)

    def _get_random_order(self):
        """
        AIの発注データが存在しないとき初期データをランダム生成する
        """
        return AIParamOrder(random.choice(list(OrderType)),
                            0,
                            0,
                            self.DEFAULT_MIN,
                            self.DEFAULT_MAX).mutation()

    def _get_order_type(self):
        if self.rate.bid > self.rate.h24:
            return OrderType.Buy
        if self.rate.h24 > self.rate.bid:
            return OrderType.Sell
        return OrderType.Wait

    @property
    def standard_tick(self):
        return self.param.get('DEFAULT_STANDARD_TICK').value


# レート取得
rate = Rate.get(currency='USDJPY', start_at=datetime.datetime.now())

# AI生成
ai = AI(rate)

# 発注
ai.order()

print ai.param

>>python genetic.py
>>OrderDone:OrderType.Sell, Limit:122.37, Stop-Limit:117.22
>>{'DEFAULT_STANDARD_TICK': VALUE:30,MIN:10,MAX:400, 'H24:1,H4:0,LOW:9,HIGH:-5,LOW_PASSED_DAY:8,HIGH_PASSED_DAY:2': ORDER-TYPE:OrderType.Sell,LIMIT:237,STOP-LIMIT:278,MIN:10,MAX:400}


SyntaxError: invalid syntax (<ipython-input-2-0959fcfbd534>, line 8)